<a href="https://colab.research.google.com/github/a24lorie/DeepLearningKeras/blob/UIMP/DL_Crear_y_entrenar_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crear y entrenar CNNs en Keras
## Ejemplo de clasificación de imágenes con CIFAR10

**NOTA IMPORTANTE**

Recuerda que estos notebook están compartidos únicamente con **permisos de lectura**, por tanto no podrás ni ejecutarlos ni modificarlos. Para poder interactuar con el notebook **debes hacer una copia del mismo en tu Drive**, usando la opción correspondiente del menú "Archivo" de *Colaboratory*.

Recuerda también que si durante la realización de la práctica tienes la sensación de que **el notebook no está funcionando bien**, puedes ir al menú "Entorno de ejecución" de *Colaboratory* y "Reiniciar el entorno de ejecución".

### 1. Importar librerías
Lo primero que vamos a hacer es cargar los módulos necesarios de la librería Keras (The Python Deep Learning library).


In [0]:
from keras import optimizers

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


### 2. Preparar datos de entrenamiento
En este ejemplo vamos a utilizar un conjunto para clasificación de imágenes denominado **CIFAR10**, que está compuesto por 50.000 imágenes de entrenamiento y 10.000 imágenes de test. Se trata de imágenes en color, de dimensiones espaciales 32x32 y etiquetadas en 10 categorías.

*En la web de Keras puedes encontrar otros conjuntos de imágenes disponibles para descarga: * https://keras.io/datasets/

El método *ImageDataGenerator()* permite generar batches de imágenes con diferentes técnicas de **data augmentation**, utilizando para ello una serie de parámetros opcionales que **no** se han considerado en este ejemplo.

*En la web de Keras puedes encontrar más información sobre este método: * https://keras.io/preprocessing/image/

In [0]:
# Cargamos los datos de CIFAR10 (entrenamiento y test)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Especificamos el numero de clases y las dimensiones de las imagenes
n_classes = 10                # numero de clases
img_width = img_height = 32   # dimensiones de la imagen

# Convertimos el vector de etiquetas en una matriz binaria para codificar las diferentes clases 
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

# Preparamos los datos, dejando una parte para validación y utilizando normalización (featurewise)
val_split = 0.2
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, validation_split=val_split)
datagen.fit(x_train)  # calcular los valores para la normalizacion (media, desviación típica)

# Generamos los batches con los datos para las tres particiones: entrenamiento, validación y test
batch_size = 256
data_train = datagen.flow(x_train, y_train, batch_size=batch_size, subset="training")
data_dev = datagen.flow(x_train, y_train, batch_size=batch_size, subset="validation")
data_test = datagen.flow(x_test, y_test, batch_size=batch_size)

### 3. Crear una CNN
El siguiente paso es crear una sencilla CNN. Para ello utilizaremos las siguientes capas:

*   Capa convolucional: *Conv2D(n_filters, kernel_size)*, siendo *n_filters* el número de filtros y *kernel_size* su tamaño. Dos de sus parámetros opcionales son:
> * *activation*: 'relu', 'sigmoid', etc. (por defecto, *activation=None*; es decir, no se utiliza función de activación).
> * *input_shape*: cuando se utiliza como primera capa del modelo, es necesario indicar las dimensiones del volumen de entrada (imagen).

*   Capa max-pooling: *MaxPooling2D()*, por defecto utiliza un tamaño de ventana 2.

*   Capa completamente conectada: *Dense(units)*, siendo *units* el número de neuronas. Uno de sus parámetros opcionales es:
> * *activation*: 'relu', 'sigmoid', 'softmax', etc.

*En la web de Keras puedes encontar información más detallada con todas las capas disponibles y los parámetros de las mismas, además de las diferentes funciones de activación.*

**Lee los comentarios del código detenidamente para entender lo que se está haciendo.**

Fíjate que la línea **model.summary()** imprime una representación en modo texto de la red y además indica el número de parámetros que se deben aprender en cada capa.



In [0]:
# Creamos un modelo secuencial, compuesto por una secuencia lineal de capas
model = Sequential()

# Añadimos dos capas convolucionales de 32 filtros (dimensiones 3x3), con ReLU como función de activación
model.add(Conv2D(32, 3, activation='relu', input_shape=(img_width,img_height,3)))
model.add(Conv2D(32, 3, activation='relu'))
# Añadimos una capa max-pooling con tamaño de ventana 2
model.add(MaxPooling2D())

# Añadimos dos capas convolucionales de 64 filtros (dimensiones 3x3), con ReLU como función de activación
model.add(Conv2D(64, 3, activation='relu'))
model.add(Conv2D(64, 3, activation='relu'))
# Añadimos una capa max-pooling con tamaño de ventana 2
model.add(MaxPooling2D())

# Flatten permite transformar el volumen de entrada en un vector
model.add(Flatten())
# Añadimos una capa completamente conectada con 512 neuronas, con ReLU como función de activación 
model.add(Dense(512, activation='relu'))
# Añadimos una última capa completamente conectada con 10 neuronas (número de clases) para obtener la salida de la red,
# utilizando para ello la función Softmax
model.add(Dense(n_classes, activation='softmax'))

# Imprimimos la representacion en modo texto del modelo 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flat

### 4. Entrenar una CNN
Una vez definida la arquitectura de la CNN, el siguiente paso es entrenar el modelo para buscar los parámetros que hagan mínima la función de pérdida. Para ello utilizaremos el método **fit_generator**, que necesita que le suministremos los datos de entrenamiento y validación, el tamaño del *batch* y el número de *epochs*. 

Finalmente, podemos evaluar el modelo utilizando el conjunto de test con el método **evaluate_generator**.

In [0]:
# Antes de entrenar el modelo, configuramos el proceso de aprendizaje
model.compile(loss='categorical_crossentropy',     # función de pérdida para problemas de clasificación multi-clase
              optimizer=optimizers.adam(lr=1e-3),  # optimizador Adam, learning rate (lr)
              metrics=['accuracy'])

# Entrenamos el modelo con los datos preparados en el punto 2
model.fit_generator(data_train,
                    epochs=6,  # numero de epochs
                    verbose=2,  # muestra informacion del error al finalizar cada epoch
                    steps_per_epoch=len(x_train)*(1-val_split)/batch_size,
                    validation_data=data_dev,
                    validation_steps=len(x_train)*val_split/batch_size)

# Por ultimo, podemos evaluar el modelo en el conjunto de test
print()
test_loss, test_acc = model.evaluate_generator(data_test,
                                               steps=len(x_test)/batch_size,
                                               verbose=1)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

Instructions for updating:
Use tf.cast instead.
Epoch 1/6
 - 10s - loss: 1.5805 - acc: 0.4302 - val_loss: 1.3019 - val_acc: 0.5343
Epoch 2/6
 - 6s - loss: 1.1426 - acc: 0.5976 - val_loss: 1.0577 - val_acc: 0.6226
Epoch 3/6
 - 6s - loss: 0.9482 - acc: 0.6673 - val_loss: 0.9730 - val_acc: 0.6626
Epoch 4/6
 - 6s - loss: 0.8164 - acc: 0.7162 - val_loss: 0.8888 - val_acc: 0.6960
Epoch 5/6
 - 6s - loss: 0.7000 - acc: 0.7579 - val_loss: 0.8106 - val_acc: 0.7196
Epoch 6/6
 - 6s - loss: 0.5901 - acc: 0.7970 - val_loss: 0.7931 - val_acc: 0.7284

40/39 [==============================] - 1s 19ms/step
test_loss: 0.8169, test_acc: 0.7230


## 5. Pruebas
Puedes probar otras configuraciones del modelo para intentar conseguir mejores resultados, e incluso definir tu propia arquitectura (otra secuencia de capas, diferentes hiperparámetros, etc.).

Para utilizar **Dropout** en el entrenamiento, es necesario añadir nuevas capas al modelo: *model.add(Dropout(rate))*, siendo *rate* el porcentaje de neuronas que se desactivan. Prueba, por ejemplo, a añadir una capa Dropout después de cada capa max-pooling.

Para generar nuevos ejemplos de entrenamiento utilizando diferentes técnicas de **data augmentation**, puedes utilizar algunos de los parámetros opcionales del método *ImageDataGenerator*.

